# Libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
import glob
from time import sleep
import json
from datetime import date 
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

spot = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id = client_id,
                                                                           client_secret= client_secret))


# Classes

In [2]:
class Band:
    def __init__(self,name: str):
        data_band = spot.search(name,type = 'artist',market='BR')['artists']['items']
        self.name = data_band[0]['name']
        self.id = data_band[0]['id']

class Albuns:
    def __init__(self,band_id: str):
        #self.data=  spot.artist_albums(band_id,album_type='album', country='BR')['items']
        self.id = band_id
        albuns = []
        for album in spot.artist_albums(band_id,country='BR')['items']:
            albuns.append((album['name'],album['id'],album['album_group']))
        self.albuns = albuns

class Album:
    def __init__(self,album_id: str):
        self.id = album_id
        data_album = spot.album(album_id,market='BR')
        self.band = data_album['artists'][0]['name']
        self.name = data_album['name']
        self.popularity = data_album['popularity']
        self.release_date = data_album['release_date']
        self.total_tracks = data_album['total_tracks']
                
class Tracks_in_Album:
    
    def __init__(self, album_id: str):
        #self.data = spot.album_tracks(album_id, market ='BR')['items']
        self.id = album_id
        tracks = []
        for track in spot.album_tracks(album_id,market='BR')['items']:
            tracks.append((track['name'],track['id']))
        self.tracks = tracks
            
class Track:
    def __init__ (self, track_id:str):
#         self.data = spot.track(track_id,market='BR')
        data_track = spot.track(track_id,market='BR')
        self.id = track_id
        self.band = data_track['artists'][0]['name']
        self.album = data_track['album']['name']
        self.name = data_track['name']
        self.popularity = data_track['popularity']
        data_features = spot.audio_features(track_id)
        self.danceability = data_features[0]['danceability']
        self.energy = data_features[0]['energy']
        self.loudness = data_features[0]['loudness']
        self.mode = data_features[0]['mode']
        self.speechiness = data_features[0]['speechiness']
        self.acousticness = data_features[0]['acousticness']
        self.instrumentalness = data_features[0]['instrumentalness']
        self.liveness = data_features[0]['liveness']
        self.valence = data_features[0]['valence']
        self.tempo = data_features[0]['tempo']
        self.duration_ms = data_features[0]['duration_ms']
        self.time_signature = data_features[0]['time_signature']
        
        
class TopTracks:
    def __init__(self,band_id:str):
        data_toptracks = spot.artist_top_tracks(band_id)['tracks']
        tracks = []
        for track in data_toptracks:
            tracks.append((track['name'],track['id']))
        self.tracks = tracks

# Functions

In [4]:
def all_songs(artist):
    if isinstance(artist,str):
        lista_de_bandas = [artist]
    if isinstance(artist,list):
        lista_de_bandas = artist
    import_tracks = []
    import_albums = []
    df_songs = []
    df_albums = []
    for banda1 in lista_de_bandas:
        banda = Band(banda1)
        banda_albums = Albuns(banda.id)

        for album in banda_albums.albuns:
#             di_a = vars(Album(album[1]))
#             di_a['album_group'] = album[2]
#             di_a['date_import'] = date.today()
#             print(di_a)

            for musica in Tracks_in_Album(album[1]).tracks:
#                 print(musica[0])
                di_s = vars(Track(musica[1]))
                di_s['date_import'] = date.today()
                import_tracks.append(di_s)
    df_songs = pd.DataFrame(import_tracks)
    df_songs.drop_duplicates(inplace=True)
    return df_songs

def all_albuns(artist):
    lista_de_bandas = [artist]
    import_albums = []
    df_albums = []
    for banda1 in lista_de_bandas:
        banda = Band(banda1)
        banda_albums = Albuns(banda.id)

        for album in banda_albums.albuns:
            di_a = vars(Album(album[1]))
            di_a['album_group'] = album[2]
            di_a['date_import'] = date.today()
#             print(di_a)
            import_albums.append(di_a)
#             for musica in Tracks_in_Album(album[1]).tracks:
#                 print(musica[0])

    df_albums = pd.DataFrame(import_albums)
    df_albums.drop_duplicates(inplace=True) 
    return df_albums

def top_tracks(artist):
    posicao = 1
    lista_de_bandas = [artist]
    import_tracks = []
    df_songs = []
    for banda1 in lista_de_bandas:
        banda = Band(banda1)
        banda_toptracks = TopTracks(banda.id)

#         print(f'Importando: {banda.name}')
        for musica in banda_toptracks.tracks:
    #         print(musica[0])
            di_s = vars(Track(musica[1]))
            di_s['date_import'] = date.today()
            di_s['position'] = posicao
            import_tracks.append(di_s)
            posicao += 1
        posicao = 1
    df_songs = pd.DataFrame(import_tracks)
    print('Finalizado')
    return df_songs

def playlist(playlist_id):
    position = 1
    lista_dict = []
    for track in spot.playlist(playlist_id)['tracks']['items']:
        dict1 = {}
        lista_artista =[]
        for artista in track['track']['artists']:
            lista_artista.append(artista['name'])
        dict1['position'] = position
        dict1['song'] = track['track']['name']
        dict1['artists'] = lista_artista
        dict1['date_import'] = today = date.today()
        position += 1
        lista_dict.append(dict1)
    df_playlist = pd.DataFrame(lista_dict)
    df_playlist.set_index('position',inplace=True)
    return df_playlist

# Extracting all songs and albuns from an artist
Querying artist name

In [6]:
songs = all_songs('vended')
songs

,id,band,album,name,popularity,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,date_import
0,3DlqHtVbVtpWBNEeI85cVh,Vended,Overall,Overall,49,0.374,0.981,-4.347,1,0.1690,0.000203,0.000010,0.0833,0.0383,100.269,232783,4,2022-11-15
1,0WATUz2EzNC0NT8w4D3HE7,Vended,Ded to Me,Ded to Me,52,0.236,0.963,-4.096,1,0.0594,0.000014,0.005760,0.2820,0.1170,100.307,224350,4,2022-11-15
2,2W3gG3gJdA6zzIwBmWD4bT,Vended,What Is It//Kill It,Antibody,49,0.398,0.992,-3.023,0,0.1760,0.001130,0.000000,0.4130,0.2320,176.973,189840,4,2022-11-15
3,2e30f4VEUwQgqcjkw7ZaIE,Vended,What Is It//Kill It,Asylum,34,0.373,0.964,-4.151,1,0.1320,0.001310,0.000164,0.3250,0.3080,140.046,172232,4,2022-11-15
4,5jfSMWpJwcWESRAMK45eeC,Vended,What Is It//Kill It,My Wrongs,44,0.392,0.985,-3.999,0,0.1840,0.001910,0.000017,0.4780,0.1490,126.035,184955,4,2022-11-15
5,7k6s7lYXnFlD8FD66HKarl,Vended,What Is It//Kill It,Bloodline,40,0.227,0.966,-3.407,1,0.0990,0.001590,0.000005,0.1160,0.2890,70.179,253952,3,2022-11-15
6,4fjmHgI9Bq1D9BZvikNDyD,Vended,What Is It//Kill It,Burn My Misery,33,0.471,0.976,-3.885,1,0.1430,0.001670,0.000019,0.1220,0.1950,141.901,318079,4,2022-11-15
7,5sUQJq1Dzwm0bLaJElLeFD,Vended,Burn My Misery,Burn My Misery,45,0.471,0.976,-3.885,1,0.1430,0.001670,0.000019,0.1220,0.1950,141.901,318079,4,2022-11-15
8,3APmM5Ib1zW8shYmzpiLCF,Vended,Burn My Misery,Asylum,28,0.373,0.964,-4.151,1,0.1320,0.001310,0.000164,0.3250,0.3080,140.046,172232,4,2022-11-15
9,2lEnc7uc39Lwrredr63OPc,Vended,Asylum,Asylum,55,0.373,0.964,-4.151,1,0.1320,0.001310,0.000164,0.3250,0.3080,140.046,172232,4,2022-11-15


In [8]:
albuns = all_albuns('vended')
albuns

,id,band,name,popularity,release_date,total_tracks,album_group,date_import
0,73OwLmHDJOZQQgQlYbqwGX,Vended,Overall,38,2022-11-02,1,single,2022-11-15
1,1XyQDiFCupKpn5E9yAtlRL,Vended,Ded to Me,41,2022-07-15,1,single,2022-11-15
2,2yJV9zQTy3D005yEBwqGjy,Vended,What Is It//Kill It,43,2021-11-12,5,single,2022-11-15
3,74DGAYxGN124ZExpyHDOSV,Vended,Burn My Misery,34,2021-10-19,2,single,2022-11-15
4,61O358weV4lViJuNItk4cT,Vended,Asylum,44,2021-09-21,1,single,2022-11-15


# Extracting TOP 10 tracks from artist
Querying artist name

In [9]:
top_tracks = top_tracks('slipknot')
top_tracks

Finalizado


,id,band,album,name,popularity,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,date_import,position
0,61mWefnWQOLf90gepjOCb3,Slipknot,Vol. 3: The Subliminal Verses,Duality,79,0.354,0.982,-3.026,0,0.1680,0.000237,0.000294,0.2040,0.194,143.772,252613,4,2022-11-15,1
1,6wqJeItl3Vc3az4ZicSQAB,Slipknot,Vol. 3: The Subliminal Verses,Before I Forget,76,0.291,0.974,-2.997,0,0.1710,0.017700,0.000005,0.8860,0.348,134.724,278600,4,2022-11-15,2
2,3RAFcUBrCNaboRXoP3S5t1,Slipknot,All Hope Is Gone (10th Anniversary),Psychosocial,75,0.568,0.981,-3.889,1,0.0887,0.002690,0.002750,0.0243,0.307,135.146,284120,4,2022-11-15,3
3,3GiJq4AQK7324mfIQbpiTf,Slipknot,.5: The Gray Chapter (Special Edition),Custer,73,0.407,0.980,-1.580,1,0.1070,0.001740,0.000112,0.1190,0.537,131.066,254467,4,2022-11-15,4
4,5mpUKTdskZea0gStWzeHUZ,Slipknot,We Are Not Your Kind,Unsainted,72,0.427,0.946,-2.188,1,0.0469,0.000118,0.025300,0.0604,0.237,101.305,260773,4,2022-11-15,5
5,15DLl1r2zi07Ssq5RT1yT0,Slipknot,Slipknot,Wait and Bleed,72,0.382,0.996,-4.119,1,0.1040,0.002080,0.000000,0.4170,0.327,93.345,147840,4,2022-11-15,6
6,5hheGdf1cb4rK0FNiedCfK,Slipknot,.5: The Gray Chapter (Special Edition),The Devil in I,72,0.398,0.939,-2.865,0,0.0648,0.005910,0.000881,0.3570,0.235,92.027,342821,4,2022-11-15,7
7,6rsoBvxrlxdmqJyGPPciyq,Slipknot,"The End, So Far",The Dying Song (Time To Sing),70,0.516,0.991,-2.587,0,0.1260,0.000467,0.000047,0.3270,0.402,95.017,203587,4,2022-11-15,8
8,2vPZ4Lklyu75zBR3SgbFNI,Slipknot,All Hope Is Gone (10th Anniversary),Snuff,64,0.544,0.690,-5.723,0,0.0425,0.012000,0.009500,0.0615,0.219,123.962,276147,4,2022-11-15,9
9,5ih5d9WJSI7Hxz0KIPJPY2,Slipknot,"The End, So Far",Yen,69,0.562,0.934,-4.264,1,0.1110,0.000833,0.015900,0.2980,0.302,137.005,283787,4,2022-11-15,10


# Extracting Playlist Songs and ranking
Using playlist ID

In [10]:
top_50_brasil = playlist('37i9dQZEVXbMXbN3EUUhlg')
top_50_brasil

,song,artists,date_import
position,,,
1,Evoque Prata,"[MC MENOR HR, MC MENOR SG, DJ ESCOBAR]",2022-11-15
2,Eu Gosto Assim - Ao Vivo,"[Gustavo Mioto, Mari Fernandez]",2022-11-15
3,Bombonzinho - Ao Vivo,"[Israel & Rodolffo, Ana Castela]",2022-11-15
4,Haja Colírio (feat. Hugo & Guilherme) - Ao Vivo,"[Guilherme & Benuto, Hugo & Guilherme]",2022-11-15
5,Casei Com a Putaria,"[Mc Paiva ZS, MC Ryan SP, Kotim]",2022-11-15
6,Assunto Delicado - Ao Vivo,"[Guilherme & Benuto, Xand Avião]",2022-11-15
7,Felina,"[WIU, MC Ryan SP]",2022-11-15
8,Pactos - Ao Vivo,"[Matheus & Kauan, Jorge & Mateus]",2022-11-15
9,Beijo de Glicose - Ao Vivo,"[Diego & Victor Hugo, Jorge & Mateus]",2022-11-15
